In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.globals import set_debug

# Variables

In [2]:
load_dotenv(override=True)

True

In [3]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)

In [5]:
file = loader.load()

# Recursive TextSplit

In [6]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [7]:
documents = recur_split.split_documents(file)

# Embbiding (OpenAI)

In [8]:
embeddings = OpenAIEmbeddings(api_key=api_key)

# Vector Store

In [9]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# Retriever

In [10]:
retriever = vectordb.as_retriever(
    search_type='mmr',
    search_kwargs=
    {
        "k": 4
    }
)

# LLM (OpenAI)

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Prompt

In [ ]:
system_message = SystemMessagePromptTemplate.from_template(
"""
Você é um assistente de extração de dados educacionais.
Sua tarefa é localizar exatamente os Códigos e Descrições que já estão presentes no contexto fornecido.

REGRAS IMPORTANTES:
- Você NÃO pode inventar nenhum conteúdo que não esteja literalmente no contexto.
- Retorne apenas os códigos e descrições exatos e completos que aparecem no contexto.
- Você pode retornar um ou mais pares (Código, Descrição), conforme os dados.
- Não corte partes das descrições.
- Não relacione o conteúdo com códigos errados.
- Se não encontrar nada, responda somente: NÃO ENCONTRADO.
"""
)

In [13]:
human_message = HumanMessagePromptTemplate.from_template(
"""
Contexto:
{context}

FORMATO OBRIGATÓRIO DE SAÍDA (copiar literalmente do contexto; repita o bloco abaixo para cada par encontrado):
Código: <código exato>
Descrição: <descrição completa>

---
Pergunta do usuário:
{question}
"""
)

In [14]:
prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Chain

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=False
)

# Question

In [16]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Experimentos Aleatórios> e o **Objetivo da Aula** é <Reconhecer>."

# Answer

In [17]:
chat_chain.invoke({"query": question})

{'query': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Experimentos Aleatórios> e o **Objetivo da Aula** é <Reconhecer>.',
 'result': 'Código: EF09MA19  \nDescrição: Reconhecer, em experimentos aleatórios, eventos independentes e dependentes e calcular a probabilidade de sua ocorrência, nos dois casos.'}